## College Recommendation System
This is an attempt to use the information at College Scorecard, the usgov data tracking system for college stats, to recommend potential target colleges for students based on their current school choices.

In [1]:
import pandas as pd
import json
import pickle

df = pd.read_csv('big_college.csv')

/Users/aaronlee/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (9,1547,1589,1595,1596,1597,1598,1601,1602,1608,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1640,1643,1648,1653,1657,1664,1665,1666,1669,1670,1674,1679,1683,1688,1689,1690,1691,1692,1697,1700,1702,1703,1704,1705,1706,1707,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1846,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df = df[df['CCSIZSET']>1]  # alternativly, use df[df['PREDDEG']>1]  (degree seeking majority)
len(df)

3673

In [3]:
#pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 4000
df.columns


Index(['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP',
       'ACCREDAGENCY', 'INSTURL', 'NPCURL', 'SCH_DEG', 'HCM2', 'MAIN',
       'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'ST_FIPS', 'REGION',
       'LOCALE', 'LOCALE2', 'LATITUDE', 'LONGITUDE', 'CCBASIC', 'CCUGPROF',
       'CCSIZSET', 'HBCU', 'PBI', 'ANNHI', 'TRIBAL', 'AANAPII', 'HSI', 'NANTI',
       'MENONLY', 'WOMENONLY', 'RELAFFIL', 'ADM_RATE', 'ADM_RATE_ALL',
       'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75',
       'SATVRMID', 'SATMTMID', 'SATWRMID', 'ACTCM25', 'ACTCM75', 'ACTEN25',
       'ACTEN75', 'ACTMT25', 'ACTMT75', 'ACTWR25', 'ACTWR75', 'ACTCMMID',
       'ACTENMID', 'ACTMTMID', 'ACTWRMID', 'SAT_AVG', 'SAT_AVG_ALL', 'PCIP01',
       'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12',
       'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23',
       'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31',
       'PCIP38', 'PCIP39

In [4]:
df = df[df['DISTANCEONLY']==0] # lose the online schools
df = df[df['PREDDEG'] == 3] # lose the 2 year schools and community colleges (lose 1600)

df.reset_index(inplace=True)
len(df)


2001

In [5]:
ident = [            
    'UNITID', # index
    'INSTNM', # for identification
    'CITY',
    'STABBR',
    'INSTURL',
    'UGDS',
    'CONTROL',
    'COSTT4_A',
    'ADM_RATE',
        ]

cat = [#'STABBR', #state
       'REGION', #midwest, northesst etc.
       'CONTROL', # public=0, private=1, private nonprofit=2  CAT
       'HBCU',
#        'PBI',
#        'ANNHI',
#        'TRIBAL',
#        'AANAPII',
#        'HSI',
#        'NANTI',
       'MENONLY',
       'WOMENONLY',
       'RELAFFIL',
      ]

cardinal = [
    'LOCALE', # how rural CARD
    'CCBASIC', # focus of school CARD
    'CCUGPROF', # focus of school CARD   
    'CCSIZSET',  # size of school
]

continuous = [
    'ADM_RATE',
    'ACTCMMID',
    'ACTENMID',
    'ACTMTMID',
    'ACTWRMID',
    'SATMTMID',
    'SATWRMID',
    'SAT_AVG',
#     'PCIP01',
#     'PCIP03', # percentage of each degree type (are we an engineering school, english, journalism etc.)
#     'PCIP04',
#     'PCIP05',
    'PCIP09', # JOURNALISM
#     'PCIP10',
#    'PCIP11', # COMPSCI
#     'PCIP12',
    'PCIP13', # EDUCATION
    'PCIP14', # ENGINEERING
#     'PCIP15',
#     'PCIP16',
#     'PCIP19',
#     'PCIP22',
#     'PCIP23',
    'PCIP24', # LIBARTS
#     'PCIP25',
#     'PCIP26',
#     'PCIP27',
#     'PCIP29',
#     'PCIP30',
#     'PCIP31',
#     'PCIP38',
#     'PCIP39',
#     'PCIP40',
#     'PCIP41',
#     'PCIP42',
#     'PCIP43',
#     'PCIP44',
#     'PCIP45',
#     'PCIP46',
#     'PCIP47',
#     'PCIP48',
#     'PCIP49',
    'PCIP50', # PERFORMING ARTS
#     'PCIP51',
#     'PCIP52',
#     'PCIP54',
    'UGDS', # student population
    'COSTT4_A', # cost attendance
    'PCTPELL', # pell grant percent
    #'C200_4', # completeion rate for 4yr students
    #'FAMINC', #family income
]

df_final = df[cardinal+continuous+continuous+cardinal+continuous+cardinal]
df_dummies = pd.get_dummies(df[cat])

df_final = pd.concat([df_final, df_dummies], axis=1)
df_final_named = df[ident]


In [6]:
pd.options.display.max_columns = 300
df_final.shape
df_final.head(50)

,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,REGION,CONTROL,HBCU,MENONLY,WOMENONLY,RELAFFIL
0,12.0,18.0,10.0,14.0,0.8986,18.0,17.0,17.0,NaN,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,0.7067,0.8986,18.0,17.0,17.0,NaN,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,0.7067,12.0,18.0,10.0,14.0,0.8986,18.0,17.0,17.0,NaN,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,0.7067,12.0,18.0,10.0,14.0,5,1,1.0,0.0,0.0,NaN
1,12.0,15.0,9.0,15.0,0.9211,25.0,27.0,23.0,NaN,555.0,NaN,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,0.3632,0.9211,25.0,27.0,23.0,NaN,555.0,NaN,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,0.3632,12.0,15.0,9.0,15.0,0.9211,25.0,27.0,23.0,NaN,555.0,NaN,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,0.3632,12.0,15.0,9.0,15.0,5,1,0.0,0.0,0.0,NaN
2,12.0,16.0,15.0,12.0,0.8087,28.0,30.0,27.0,NaN,565.0,NaN,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,0.2698,0.8087,28.0,30.0,27.0,NaN,565.0,NaN,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,0.2698,12.0,16.0,15.0,12.0,0.8087,28.0,30.0,27.0,NaN,565.0,NaN,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,0.2698,12.0,16.0,15.0,12.0,5,1,0.0,0.0,0.0,NaN
3,12.0,19.0,10.0,14.0,0.9774,18.0,17.0,17.0,NaN,465.0,NaN,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,0.7448,0.9774,18.0,17.0,17.0,NaN,465.0,NaN,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,0.7448,12.0,19.0,10.0,14.0,0.9774,18.0,17.0,17.0,NaN,465.0,NaN,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,0.7448,12.0,19.0,10.0,14.0,5,1,1.0,0.0,0.0,NaN
4,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,0.1802,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,0.1802,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,0.1802,12.0,15.0,14.0,16.0,5,1,0.0,0.0,0.0,NaN
5,31.0,22.0,5.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.3333,0.0000,0.0461,0.0303,2732.0,NaN,0.4223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.3333,0.0000,0.0461,0.0303,2732.0,NaN,0.4223,31.0,22.0,5.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.3333,0.0000,0.0461,0.0303,2732.0,NaN,0.4223,31.0,22.0,5.0,9.0,5,1,0.0,0.0,0.0,NaN
6,12.0,18.0,9.0,13.0,0.9281,21.0,21.0,19.0,NaN,525.0,NaN,1084.0,0.0339,0.0854,0.0000,0.0027,0.0298,4529.0,18291.0,0.4584,0.9281,21.0,21.0,19.0,NaN,525.0,NaN,1084.0,0.0339,0.0854,0.0000,0.0027,0.0298,4529.0,18291.0,0.4584,12.0,18.0,9.0,13.0,0.9281,21.0,21.0,19.0,NaN,525.0,NaN,1084.0,0.0339,0.0854,0.0000,0.0027,0.0298,4529.0,18291.0,0.4584,12.0,18.0,9.0,13.0,5,1,0.0,0.0,0.0,NaN
7,13.0,15.0,14.0,15.0,0.7543,28.0,29.0,26.0,8.0,615.0,570.0,1289.0,0.0566,0.0602,0.1911,0.0000,0.0296,24147.0,31282.0,0.1537,0.7543,28.0,29.0,26.0,8.0,615.0,570.0,1289.0,0.0566,0.0602,0.1911,0.0000,0.0296,24147.0,31282.0,0.1537,13.0,15.0,14.0,15.0,0.7543,28.0,29.0,26.0,8.0,615.0,570.0,1289.0,0.0566,0.0602,0.1911,0.0000,0.0296,24147.0,31282.0,0.1537,13.0,15.0,14.0,15.0,5,1,0.0,0.0,0.0,NaN
8,12.0,21.0,14.0,11.0,0.5666,26.0,27.0,23.0,NaN,560.0,NaN,1232.0,0.0284,0.0355,0.0248,0.0000,0.1099,1265.0,52176.0,0.2214,0.5666,26.0,27.0,23.0,NaN,560.0,NaN,1232.0,0.0284,0.0355,0.0248,0.0000,0.1099,1265.0,52176.0,0.2214,12.0,21.0,14.0,11.0,0.5666,26.0,27.0,23.0,NaN,560.0,NaN,1232.0,0.0284,0.0355,0.0248,0.0000,0.1099,1265.0,52176.0,0.2214,12.0,21.0,14.0,11.0,5,2,0.0,0.0,0.0,71.0
9,12.0,20.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,

In [7]:
#df_final.fillna('')


In [8]:
pd.options.display.max_rows = 200
df_final.isna().sum()
df_final.fillna(0, inplace=True)
# df_final['FAMINC'] = df_final['FAMINC'].isna
# df_final['FAMINC'] = df_final['FAMINC'].astype(float)


In [9]:
df_final.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 66 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LOCALE     2001 non-null   float64
 1   CCBASIC    2001 non-null   float64
 2   CCUGPROF   2001 non-null   float64
 3   CCSIZSET   2001 non-null   float64
 4   ADM_RATE   2001 non-null   float64
 5   ACTCMMID   2001 non-null   float64
 6   ACTENMID   2001 non-null   float64
 7   ACTMTMID   2001 non-null   float64
 8   ACTWRMID   2001 non-null   float64
 9   SATMTMID   2001 non-null   float64
 10  SATWRMID   2001 non-null   float64
 11  SAT_AVG    2001 non-null   float64
 12  PCIP09     2001 non-null   float64
 13  PCIP13     2001 non-null   float64
 14  PCIP14     2001 non-null   float64
 15  PCIP24     2001 non-null   float64
 16  PCIP50     2001 non-null   float64
 17  UGDS       2001 non-null   float64
 18  COSTT4_A   2001 non-null   float64
 19  PCTPELL    2001 non-null   float64
 20  ADM_RATE

In [10]:
df_final['ACTWRMID'] = df_final['ACTWRMID'].fillna(6)
df_final['ACTWRMID'] = df_final['ACTWRMID'].fillna(6)

# df_final['RELAFFIL'].unique()
# df_final['RELAFFIL'] = df_final['RELAFFIL'].fillna(0)

#df_final.max()
df_final.isna().sum().max()


0

## Normalize my data


In [11]:
import scipy
import pandas as pd
import numpy as np
from sklearn import preprocessing


scaler = preprocessing.QuantileTransformer()

scaled_data = scaler.fit_transform(df_final)
df_scaled = pd.DataFrame(scaled_data, columns= df_final.columns)

# # WEIGHTING
# weights = [
#             ('ADM_RATE', 50),
#             ('REGION', 1000),
#             ('UGDS', 50),
#             ('CONTROL', 5),
#             ('COSTT4_A', 5),
#             ('MENONLY', 50),
#             ('WOMENONLY', 50),
#             ('HBCU', 100),
#             ('CCBASIC', 10),
#             ('CCUGPROF', 10),
#             ('CCSIZSET', 20),
#             ('ACTCMMID', 50),
#             ('SATMTMID', 50),
#             ('SATWRMID', 50),
#             ('ACTENMID', 50),
#             ('ACTMTMID', 50),
#             ('ACTWRMID', 50)
#             ]

# for col, weight in weights:
#     df_scaled[col] = df_scaled[col] * weight
    
df_scaled.head()



,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,REGION,CONTROL,HBCU,MENONLY,WOMENONLY,RELAFFIL
0,0.318819,0.282282,0.303303,0.818318,0.885059,0.405405,0.440440,0.436436,0.000000,0.422923,0.664541,0.393693,0.000000,0.752000,0.914997,0.802803,0.523524,0.765698,0.327597,0.938034,0.885059,0.405405,0.440440,0.436436,0.000000,0.422923,0.664541,0.393693,0.000000,0.752000,0.914997,0.802803,0.523524,0.765698,0.327597,0.938034,0.318819,0.282282,0.303303,0.818318,0.885059,0.405405,0.440440,0.436436,0.000000,0.422923,0.664541,0.393693,0.000000,0.752000,0.914997,0.802803,0.523524,0.765698,0.327597,0.938034,0.318819,0.282282,0.303303,0.818318,0.641141,0.0,1.0,0.0,0.0,0.0
1,0.318819,0.000000,0.234234,0.886887,0.915002,0.822823,0.888388,0.777778,0.000000,0.728228,0.000000,0.862863,0.667668,0.756256,0.878567,0.765503,0.633634,0.911940,0.384774,0.478505,0.915002,0.822823,0.888388,0.777778,0.000000,0.728228,0.000000,0.862863,0.667668,0.756256,0.878567,0.765503,0.633634,0.911940,0.384774,0.478505,0.318819,0.000000,0.234234,0.886887,0.915002,0.822823,0.888388,0.777778,0.000000,0.728228,0.000000,0.862863,0.667668,0.756256,0.878567,0.765503,0.633634,0.911940,0.384774,0.478505,0.318819,0.000000,0.234234,0.886887,0.641141,0.0,0.0,0.0,0.0,0.0
2,0.318819,0.097598,1.000000,0.662162,0.767936,0.916416,0.947447,0.923924,0.000000,0.765766,0.000000,0.934329,0.510511,0.508503,0.975978,0.000000,0.672673,0.840909,0.354265,0.256256,0.767936,0.916416,0.947447,0.923924,0.000000,0.765766,0.000000,0.934329,0.510511,0.508503,0.975978,0.000000,0.672673,0.840909,0.354265,0.256256,0.318819,0.097598,1.000000,0.662162,0.767936,0.916416,0.947447,0.923924,0.000000,0.765766,0.000000,0.934329,0.510511,0.508503,0.975978,0.000000,0.672673,0.840909,0.354265,0.256256,0.318819,0.097598,1.000000,0.662162,0.641141,0.0,0.0,0.0,0.0,0.0
3,0.318819,0.411912,0.303303,0.818318,0.971039,0.405405,0.440440,0.436436,0.000000,0.422923,0.000000,0.400400,0.943988,0.892957,0.000000,0.000000,0.802369,0.722200,0.290843,0.949956,0.971039,0.405405,0.440440,0.436436,0.000000,0.422923,0.000000,0.400400,0.943988,0.892957,0.000000,0.000000,0.802369,0.722200,0.290843,0.949956,0.318819,0.411912,0.303303,0.818318,0.971039,0.405405,0.440440,0.436436,0.000000,0.422923,0.000000,0.400400,0.943988,0.892957,0.000000,0.000000,0.802369,0.722200,0.290843,0.949956,0.318819,0.411912,0.303303,0.818318,0.641141,0.0,1.0,0.0,0.0,0.0
4,0.318819,0.000000,0.863864,0.946446,0.435027,0.896396,0.930931,0.876376,0.941942,0.822823,0.892392,0.895943,0.954381,0.720721,0.913275,0.000000,0.534034,0.987119,0.505473,0.104270,0.435027,0.896396,0.930931,0.876376,0.941942,0.822823,0.892392,0.895943,0.954381,0.720721,0.913275,0.000000,0.534034,0.987119,0.505473,0.104270,0.318819,0.000000,0.863864,0.946446,0.435027,0.896396,0.930931,0.876376,0.941942,0.822823,0.892392,0.895943,0.954381,0.720721,0.913275,0.000000,0.534034,0.987119,0.505473,0.104270,0.318819,0.000000,0.863864,0.946446,0.641141,0.0,0.0,0.0,0.0,0.0


In [18]:
df_scaled.isna().sum()
df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 66 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LOCALE     2001 non-null   float64
 1   CCBASIC    2001 non-null   float64
 2   CCUGPROF   2001 non-null   float64
 3   CCSIZSET   2001 non-null   float64
 4   ADM_RATE   2001 non-null   float64
 5   ACTCMMID   2001 non-null   float64
 6   ACTENMID   2001 non-null   float64
 7   ACTMTMID   2001 non-null   float64
 8   ACTWRMID   2001 non-null   float64
 9   SATMTMID   2001 non-null   float64
 10  SATWRMID   2001 non-null   float64
 11  SAT_AVG    2001 non-null   float64
 12  PCIP09     2001 non-null   float64
 13  PCIP13     2001 non-null   float64
 14  PCIP14     2001 non-null   float64
 15  PCIP24     2001 non-null   float64
 16  PCIP50     2001 non-null   float64
 17  UGDS       2001 non-null   float64
 18  COSTT4_A   2001 non-null   float64
 19  PCTPELL    2001 non-null   float64
 20  ADM_RATE

In [26]:
import pickle
pickle.dump( df_scaled, open( "scaled_df.pkl", "wb" ))
pickle.dump(df_final_named, open('df_final_names.pkl', 'wb'))
df_final_named.isna().sum()
df_final_named.fillna('No data', inplace=True)
df_final_named.isna().sum()


/Users/aaronlee/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


UNITID      0
INSTNM      0
CITY        0
STABBR      0
INSTURL     0
UGDS        0
CONTROL     0
COSTT4_A    0
ADM_RATE    0
dtype: int64

# Run a test


In [27]:
college_name = 'DePaul University'
college_id = df[df['INSTNM']==college_name] # test it out
college_id = college_id.index.to_list()[0]


test_college = df_scaled.iloc[[college_id]]
#test_college_scaled = scaler.transform(test_college)
#print(test_college_scaled)



ary = scipy.spatial.distance.cdist(df_scaled, test_college, metric='euclidean')

results = df_final.copy()
results['dist'] = ary


results = pd.concat([df_final_named, results], axis=1)
results

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,REGION,CONTROL,HBCU,MENONLY,WOMENONLY,RELAFFIL,dist
0,100654,Alabama A & M University,Normal,AL,www.aamu.edu/,4990.0,1,22489.0,0.8986,12.0,18.0,10.0,14.0,0.8986,18.0,17.0,17.0,0.0,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,0.7067,0.8986,18.0,17.0,17.0,0.0,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,0.7067,12.0,18.0,10.0,14.0,0.8986,18.0,17.0,17.0,0.0,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,0.7067,12.0,18.0,10.0,14.0,5,1,1.0,0.0,0.0,0.0,3.867506
1,100663,University of Alabama at Birmingham,Birmingham,AL,https://www.uab.edu,13186.0,1,24347.0,0.9211,12.0,15.0,9.0,15.0,0.9211,25.0,27.0,23.0,0.0,555.0,0.0,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,0.3632,0.9211,25.0,27.0,23.0,0.0,555.0,0.0,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,0.3632,12.0,15.0,9.0,15.0,0.9211,25.0,27.0,23.0,0.0,555.0,0.0,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,0.3632,12.0,15.0,9.0,15.0,5,1,0.0,0.0,0.0,0.0,4.010619
2,100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,7458.0,1,23441.0,0.8087,12.0,16.0,15.0,12.0,0.8087,28.0,30.0,27.0,0.0,565.0,0.0,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,0.2698,0.8087,28.0,30.0,27.0,0.0,565.0,0.0,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,0.2698,12.0,16.0,15.0,12.0,0.8087,28.0,30.0,27.0,0.0,565.0,0.0,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,0.2698,12.0,16.0,15.0,12.0,5,1,0.0,0.0,0.0,0.0,4.560372
3,100724,Alabama State University,Montgomery,AL,www.alasu.edu,3903.0,1,21476.0,0.9774,12.0,19.0,10.0,14.0,0.9774,18.0,17.0,17.0,0.0,465.0,0.0,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,0.7448,0.9774,18.0,17.0,17.0,0.0,465.0,0.0,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,0.7448,12.0,19.0,10.0,14.0,0.9774,18.0,17.0,17.0,0.0,465.0,0.0,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,0.7448,12.0,19.0,10.0,14.0,5,1,1.0,0.0,0.0,0.0,3.323565
4,100751,The University of Alabama,Tuscaloosa,AL,www.ua.edu/,32177.0,1,29424.0,0.5906,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,0.1802,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,0.1802,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,0.1802,12.0,15.0,14.0,16.0,5,1,0.0,0.0,0.0,0.0,4.917209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,490513,Bais Medrash Mayan Hatorah,Lakewood,NJ,https://baismedrashmayanhatorah.com,69.0,2,16050.0,0.6471,13.0,24.0,11.0,8.0,0.6471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,69.0,16050.0,0.2075,0.6471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,69.0,16050.0,0.2075,13.0,24.0,11.0,8.0,0.6471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,69.0,16050.0,0.2075,13.0,24.0,11.0,8.0,2,2,0.0,0.0,0.0,0.0,4.072601
1997,490771,Purdue University Global-St. Louis,St. Louis,MO,https://www.purdueglobal.edu/campus-locations/...,232.0,1,No data,No data,21.0,22.0,5.0,6.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.00

dist_df is the wrong size?????

In [14]:
closest = results.sort_values(by='dist')
closest.head(50)

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,REGION,CONTROL,HBCU,MENONLY,WOMENONLY,RELAFFIL,dist
335,144740,DePaul University,Chicago,IL,https://www.depaul.edu,14280.0,2,54439.0,0.6753,11.0,16.0,13.0,15.0,0.6753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1318,0.0329,0.0000,0.0850,0.0859,14280.0,54439.0,0.3257,0.6753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1318,0.0329,0.0000,0.0850,0.0859,14280.0,54439.0,0.3257,11.0,16.0,13.0,15.0,0.6753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1318,0.0329,0.0000,0.0850,0.0859,14280.0,54439.0,0.3257,11.0,16.0,13.0,15.0,3,2,0.0,0.0,0.0,30.0,0.000000
1281,213367,La Salle University,Philadelphia,PA,www.lasalle.edu,3766.0,2,41961.0,0.8118,11.0,18.0,12.0,13.0,0.8118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0739,0.0443,0.0000,0.0614,0.0136,3766.0,41961.0,0.4193,0.8118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0739,0.0443,0.0000,0.0614,0.0136,3766.0,41961.0,0.4193,11.0,18.0,12.0,13.0,0.8118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0739,0.0443,0.0000,0.0614,0.0136,3766.0,41961.0,0.4193,11.0,18.0,12.0,13.0,2,2,0.0,0.0,0.0,30.0,1.212090
333,144281,Columbia College Chicago,Chicago,IL,www.colum.edu,6496.0,2,42413.0,0.8675,11.0,19.0,11.0,13.0,0.8675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1294,0.0042,0.0000,0.0006,0.6309,6496.0,42413.0,0.3464,0.8675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1294,0.0042,0.0000,0.0006,0.6309,6496.0,42413.0,0.3464,11.0,19.0,11.0,13.0,0.8675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1294,0.0042,0.0000,0.0006,0.6309,6496.0,42413.0,0.3464,11.0,19.0,11.0,13.0,3,2,0.0,0.0,0.0,0.0,1.406506
1291,213987,Mercyhurst University,Erie,PA,www.mercyhurst.edu,2374.0,2,49928.0,0.5981,13.0,19.0,12.0,11.0,0.5981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0200,0.0381,0.0000,0.0127,0.0690,2374.0,49928.0,0.2904,0.5981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0200,0.0381,0.0000,0.0127,0.0690,2374.0,49928.0,0.2904,13.0,19.0,12.0,11.0,0.5981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0200,0.0381,0.0000,0.0127,0.0690,2374.0,49928.0,0.2904,13.0,19.0,12.0,11.0,2,2,0.0,0.0,0.0,30.0,1.587018
1097,199847,Wake Forest University,Winston-Salem,NC,www.wfu.edu,5225.0,2,69090.0,0.2940,12.0,16.0,14.0,14.0,0.2940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0812,0.0067,0.0000,0.0042,0.0193,5225.0,69090.0,0.0972,0.2940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0812,0.0067,0.0000,0.0042,0.0193,5225.0,69090.0,0.0972,12.0,16.0,14.0,14.0,0.2940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0812,0.0067,0.0000,0.0042,0.0193,5225.0,69090.0,0.0972,12.0,16.0,14.0,14.0,5,2,0.0,0.0,0.0,0.0,1.638513
254,136950,Rollins College,Winter Park,FL,www.rollins.edu,2571.0,2,65487.0,0.6664,21.0,18.0,12.0,11.0,0.6664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1301,0.0151,0.0000,0.0091,0.0666,2571.0,65487.0,0.2829,0.6664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1301,0.0151,0.0000,0.0091,0.0666,2571.0,65487.0,0.2829,21.0,18.0,12.0,11.0,0.6664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1301,0.0151,0.0000,0.0091,0.0666,2571.0,65487.0,0.2829,21.0,18.0,12.0,11.0,5,2,0.0,0.0,0.0,0.0,1.657879
206,130776,Western Connecticut State University,Danbury,CT,www.wcsu.edu,4872.0,1,25198.0,0.7924,13.0,19.0,13.0,13.0,0.7924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0698,0.0349,0.0000,0.0148,0.0793,4872.0,25198.0,0.3461,0.7924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0698,0.0349,0.0000,0.0148,0.0793,4872.0,25198.0,0.3461,13.0,19.0,13.0,13.0,0.7924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0698,0.0349,0.0000,0.0148,0.0793,4872.0,25198.0,0.3461,13.0,19.0,13.0,13.0,1,1,0.0,0.0,0.0,0.0,1.703607
948,191968,Ithaca College,Ithaca,NY,www.ithaca.edu,6072.0,2,61316.0,0.6854,23.0,18.0,12.0,14.0,0.6854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2523,0.0347,0.0000,0.0051,0.1685,6072.0,61316.0,0.1981,0.6854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25

In [15]:
colleges = list(df_final_named['INSTNM'])
colleges

['Alabama A & M University',
 'University of Alabama at Birmingham',
 'University of Alabama in Huntsville',
 'Alabama State University',
 'The University of Alabama',
 'Athens State University',
 'Auburn University at Montgomery',
 'Auburn University',
 'Birmingham-Southern College',
 'South University-Montgomery',
 'Faulkner University',
 'Huntingdon College',
 'Heritage Christian University',
 'Jacksonville State University',
 'Judson College',
 'University of West Alabama',
 'Miles College',
 'University of Mobile',
 'University of Montevallo',
 'University of North Alabama',
 'Oakwood University',
 'Samford University',
 'Selma University',
 'University of South Alabama',
 'Spring Hill College',
 'Stillman College',
 'Talladega College',
 'Troy University',
 'Tuskegee University',
 'United States Sports Academy',
 'University of Alaska Anchorage',
 'Alaska Bible College',
 'University of Alaska Fairbanks',
 'Alaska Pacific University',
 'Arizona State University-Tempe',
 'Universi

In [16]:
my_json = df_final_named['INSTNM'].to_json()
my_json

'{"0":"Alabama A & M University","1":"University of Alabama at Birmingham","2":"University of Alabama in Huntsville","3":"Alabama State University","4":"The University of Alabama","5":"Athens State University","6":"Auburn University at Montgomery","7":"Auburn University","8":"Birmingham-Southern College","9":"South University-Montgomery","10":"Faulkner University","11":"Huntingdon College","12":"Heritage Christian University","13":"Jacksonville State University","14":"Judson College","15":"University of West Alabama","16":"Miles College","17":"University of Mobile","18":"University of Montevallo","19":"University of North Alabama","20":"Oakwood University","21":"Samford University","22":"Selma University","23":"University of South Alabama","24":"Spring Hill College","25":"Stillman College","26":"Talladega College","27":"Troy University","28":"Tuskegee University","29":"United States Sports Academy","30":"University of Alaska Anchorage","31":"Alaska Bible College","32":"University of Al

In [100]:
# only if you are rewriting colleges
with open('../colleges.txt', 'w') as outfile:
    json.dump(my_json, outfile)

In [101]:
df_final_named

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE
0,100654,Alabama A & M University,Normal,AL,www.aamu.edu/,4990.0,1,22489.0,0.8986
1,100663,University of Alabama at Birmingham,Birmingham,AL,https://www.uab.edu,13186.0,1,24347.0,0.9211
2,100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,7458.0,1,23441.0,0.8087
3,100724,Alabama State University,Montgomery,AL,www.alasu.edu,3903.0,1,21476.0,0.9774
4,100751,The University of Alabama,Tuscaloosa,AL,www.ua.edu/,32177.0,1,29424.0,0.5906
...,...,...,...,...,...,...,...,...,...
1996,490513,Bais Medrash Mayan Hatorah,Lakewood,NJ,https://baismedrashmayanhatorah.com,69.0,2,16050.0,0.6471
1997,490771,Purdue University Global-St. Louis,St. Louis,MO,https://www.purdueglobal.edu/campus-locations/...,232.0,1,NaN,NaN
1998,490805,Purdue University Northwest,Hammond,IN,www.pnw.edu,8244.0,1,16949.0,0.3108
1999,490850,East-West University,Keshena,WI,www.eastwest.edu,11.0,2,40100.0,NaN


In [102]:
df_scaled

,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,PCTPELL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,REGION,CONTROL,HBCU,MENONLY,WOMENONLY,RELAFFIL
0,0.318819,0.282282,0.303303,0.818318,0.885059,0.405405,0.440440,0.436436,0.000000,0.422923,0.664541,0.393693,0.000000,0.752000,0.914997,0.802803,0.523524,0.765698,0.327597,0.938034,0.885059,0.405405,0.440440,0.436436,0.000000,0.422923,0.664541,0.393693,0.000000,0.752000,0.914997,0.802803,0.523524,0.765698,0.327597,0.938034,0.318819,0.282282,0.303303,0.818318,0.885059,0.405405,0.440440,0.436436,0.000000,0.422923,0.664541,0.393693,0.000000,0.752000,0.914997,0.802803,0.523524,0.765698,0.327597,0.938034,0.318819,0.282282,0.303303,0.818318,0.641141,0.000000,1.0,0.0,0.0,0.0
1,0.318819,0.000000,0.234234,0.886887,0.915002,0.822823,0.888388,0.777778,0.000000,0.728228,0.000000,0.862863,0.667668,0.756256,0.878567,0.765503,0.633634,0.911940,0.384774,0.478505,0.915002,0.822823,0.888388,0.777778,0.000000,0.728228,0.000000,0.862863,0.667668,0.756256,0.878567,0.765503,0.633634,0.911940,0.384774,0.478505,0.318819,0.000000,0.234234,0.886887,0.915002,0.822823,0.888388,0.777778,0.000000,0.728228,0.000000,0.862863,0.667668,0.756256,0.878567,0.765503,0.633634,0.911940,0.384774,0.478505,0.318819,0.000000,0.234234,0.886887,0.641141,0.000000,0.0,0.0,0.0,0.0
2,0.318819,0.097598,1.000000,0.662162,0.767936,0.916416,0.947447,0.923924,0.000000,0.765766,0.000000,0.934329,0.510511,0.508503,0.975978,0.000000,0.672673,0.840909,0.354265,0.256256,0.767936,0.916416,0.947447,0.923924,0.000000,0.765766,0.000000,0.934329,0.510511,0.508503,0.975978,0.000000,0.672673,0.840909,0.354265,0.256256,0.318819,0.097598,1.000000,0.662162,0.767936,0.916416,0.947447,0.923924,0.000000,0.765766,0.000000,0.934329,0.510511,0.508503,0.975978,0.000000,0.672673,0.840909,0.354265,0.256256,0.318819,0.097598,1.000000,0.662162,0.641141,0.000000,0.0,0.0,0.0,0.0
3,0.318819,0.411912,0.303303,0.818318,0.971039,0.405405,0.440440,0.436436,0.000000,0.422923,0.000000,0.400400,0.943988,0.892957,0.000000,0.000000,0.802369,0.722200,0.290843,0.949956,0.971039,0.405405,0.440440,0.436436,0.000000,0.422923,0.000000,0.400400,0.943988,0.892957,0.000000,0.000000,0.802369,0.722200,0.290843,0.949956,0.318819,0.411912,0.303303,0.818318,0.971039,0.405405,0.440440,0.436436,0.000000,0.422923,0.000000,0.400400,0.943988,0.892957,0.000000,0.000000,0.802369,0.722200,0.290843,0.949956,0.318819,0.411912,0.303303,0.818318,0.641141,0.000000,1.0,0.0,0.0,0.0
4,0.318819,0.000000,0.863864,0.946446,0.435027,0.896396,0.930931,0.876376,0.941942,0.822823,0.892392,0.895943,0.954381,0.720721,0.913275,0.000000,0.534034,0.987119,0.505473,0.104270,0.435027,0.896396,0.930931,0.876376,0.941942,0.822823,0.892392,0.895943,0.954381,0.720721,0.913275,0.000000,0.534034,0.987119,0.505473,0.104270,0.318819,0.000000,0.863864,0.946446,0.435027,0.896396,0.930931,0.876376,0.941942,0.822823,0.892392,0.895943,0.954381,0.720721,0.913275,0.000000,0.534034,0.987119,0.505473,0.104270,0.318819,0.000000,0.863864,0.946446,0.641141,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.444444,0.831331,0.433433,0.258759,0.511508,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054054,0.141923,0.144025,0.511508,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054054,0.141923,0

In [103]:
df_final_named.iloc[290]

UNITID                     139959
INSTNM      University of Georgia
CITY                       Athens
STABBR                         GA
INSTURL               www.uga.edu
UGDS                      29474.0
CONTROL                         1
COSTT4_A                  26252.0
ADM_RATE                   0.4863
Name: 290, dtype: object

In [104]:
# Extract stats
def get_index(college):
    college_id = df_final_named[df_final_named['INSTNM'] == 'University of Georgia']  # test it out
    college_id = college_id.index.to_list()[0]
    return college_id

i = get_index(college_name)
stats = df_final_named.iloc[i]
stats['INSTNM']

'University of Georgia'